In [ ]:
import torch
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd

from constants import (
    TRAIN_DATA_CSV,
    TEST_DATA_CSV,
    TEST_IMAGE_FEATURES_PATH,
    TEST_TEXT_FEATURES_PATH,
    TRAIN_IMAGE_FEATURES_PATH,
    TRAIN_TEXT_FEATURES_PATH,
    MULTIMODAL_NO_ENCODER_MODEL_PATH,
)
from core.src.utils.metrics import compute_metrics
from core.src.utils.plots import plot_loss_and_metrics, print_metrics_table

In [ ]:
df_train = pd.read_csv(TRAIN_DATA_CSV, dtype={"unique_id": str})
df_test = pd.read_csv(TEST_DATA_CSV, dtype={"unique_id": str})
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
STRUCTURED_COLUMNS = [
    "km",
    "putere",
    "capacitate cilindrica",
    "anul producției",
    "marca",
    "model",
    "combustibil",
    "tip caroserie",
    "firma",
    "is_automatic",
]

train_structured_data = df_train[STRUCTURED_COLUMNS].to_numpy()
test_structured_data = df_test[STRUCTURED_COLUMNS].to_numpy()

train_targets = df_train["price"].to_numpy()
test_targets = df_test["price"].to_numpy()

train_structured_data.shape, test_structured_data.shape, train_targets.shape, test_targets.shape

In [ ]:
train_image_features = np.load(TRAIN_IMAGE_FEATURES_PATH)
test_image_features = np.load(TEST_IMAGE_FEATURES_PATH)

train_text_features = np.load(TRAIN_TEXT_FEATURES_PATH)
test_text_features = np.load(TEST_TEXT_FEATURES_PATH)

len(train_image_features), len(train_text_features), len(test_image_features), len(test_text_features)

In [ ]:
class MultimodalDataset(Dataset):
    def __init__(self, image_features, text_features, structured_data, targets):
        self.image_features = image_features
        self.text_features = text_features
        self.structured_data = structured_data
        self.targets = targets

    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        # Use the entire sequence representation
        # image = torch.tensor(self.image_features[idx], dtype=torch.float32)
        # text = torch.tensor(self.text_features[idx], dtype=torch.float32)

        # Use only the [CLS] token representation
        image = torch.tensor([self.image_features[idx][0]], dtype=torch.float32)
        text = torch.tensor([self.text_features[idx][0]], dtype=torch.float32)

        structured = torch.tensor(self.structured_data[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)

        features = torch.cat([image, text, structured])

        return features, target


train_dataset = MultimodalDataset(train_image_features, train_text_features, train_structured_data, train_targets)
test_dataset = MultimodalDataset(test_image_features, test_text_features, test_structured_data, test_targets)

BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

trainSteps = len(train_loader.dataset) // BATCH_SIZE
testSteps = len(test_loader.dataset) // BATCH_SIZE
history = {"train_loss": [], "test_loss": []}
metrics_history = {"train_mae": [], "test_mae": [], "train_rmse": [], "test_rmse": [], "train_r2": [], "test_r2": [], "train_mse": [], "test_mse": []}

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # self.fc1 = nn.Linear(1994, 512)
        self.fc1 = nn.Linear(12, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)

        return x


model = Net().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=0.5, min_lr=1e-9, patience=50)

In [ ]:
best_val_loss = float("inf")
for epoch in range(2000):
    model.train()
    train_loss = 0.0
    all_train_predictions = []
    all_train_ground_truths = []

    for features, target in train_loader:
        features = features.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, target.view(-1, 1))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * features.size(0)
        all_train_predictions.extend(outputs.cpu().detach().numpy())
        all_train_ground_truths.extend(target.view(-1, 1).cpu().detach().numpy())

    train_metrics = compute_metrics(all_train_predictions, all_train_ground_truths)
    metrics_history["train_mae"].append(train_metrics["MAE"])
    metrics_history["train_mse"].append(train_metrics["MSE"])
    metrics_history["train_rmse"].append(train_metrics["RMSE"])
    metrics_history["train_r2"].append(train_metrics["R2"])

    # Validation loop (optional, but recommended)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        test_loss = 0.0
        all_test_predictions = []
        all_test_ground_truths = []
        for features, target in test_loader:
            features = features.to(device)
            target = target.to(device)
            outputs = model(features)
            loss = criterion(outputs, target.view(-1, 1))  # Ensure 'target' is defined for test data
            test_loss += loss.item() * features.size(0)
            all_test_predictions.extend(outputs.cpu().detach().numpy())
            all_test_ground_truths.extend(target.view(-1, 1).cpu().detach().numpy())

        test_metrics = compute_metrics(all_test_predictions, all_test_ground_truths)
        metrics_history["test_mae"].append(test_metrics["MAE"])
        metrics_history["test_mse"].append(test_metrics["MSE"])
        metrics_history["test_rmse"].append(test_metrics["RMSE"])
        metrics_history["test_r2"].append(test_metrics["R2"])

        train_loss /= len(train_loader.dataset)
        test_loss /= len(test_loader.dataset)

        history["test_loss"].append(test_loss)
        history["train_loss"].append(train_loss)

        scheduler.step(test_loss)
        print(scheduler.get_last_lr())

        print(f"Epoch {epoch + 1}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}")
        print("Train Metrics: ", train_metrics)
        print("Test Metrics: ", test_metrics)

        if test_loss < best_val_loss:
            best_val_loss = test_loss
            torch.save(model.state_dict(), MULTIMODAL_NO_ENCODER_MODEL_PATH)
            print(f"Epoch {epoch + 1}: New best test loss: {best_val_loss}")

In [ ]:
plot_loss_and_metrics(history, metrics_history, SLICE_START=20)
print_metrics_table(metrics_history)